In [40]:
import os
import pathlib

import pandas as pd
import pypsa
import numpy as np
import plotly.express as px
import xarray as xr
import geopandas as gpd
import plotly.graph_objects as go
import shapely.geometry
import math
import yaml
import hvplot.pandas
import holoviews as hv
from bokeh.models import HoverTool  
from bokeh.plotting import figure, show
from holoviews import Store
import networkx as nx
import hvplot.networkx as hvnx
from shapely.geometry import Point, LineString, shape


Store.add_style_opts(hv.Bars, ['level'], backend='bokeh')


In [41]:
n=pypsa.Network("../data/test_data/elec_s_10_ec_lcopt_Co2L-4H.nc")

INFO:pypsa.io:Imported network elec_s_10_ec_lcopt_Co2L-4H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [42]:
model_nice_name =pd.DataFrame(n.carriers["nice_name"])
model_nice_name

,nice_name
Carrier,
CCGT,Combined-Cycle Gas
biomass,Biomass
offwind-ac,Offshore Wind (AC)
geothermal,Geothermal
coal,Coal
oil,Oil
solar,Solar
OCGT,Open-Cycle Gas
nuclear,Nuclear


In [43]:
def load_regions():
    fn = "../data/test_data/regions_onshore_elec_s_10.geojson"
    gdf = gpd.read_file(fn)
    gdf.geometry = gdf.to_crs(3035).geometry.simplify(1000).to_crs(4326)
    return gdf

In [44]:
gpd_bus_regions = load_regions()
gpd_bus_regions

,name,x,y,country,geometry
0,MA0 0,-4.497494,32.115008,MA,"POLYGON ((-3.74821 31.36056, -3.76156 31.31018..."
1,MA0 1,-9.393582,30.564976,MA,"MULTIPOLYGON (((-10.57073 28.10503, -10.27730 ..."
2,MA0 2,-5.884074,34.243265,MA,"MULTIPOLYGON (((-6.42646 33.26058, -6.13605 33..."
3,MA0 3,-6.568445,32.124369,MA,"POLYGON ((-7.70016 30.91545, -7.63946 31.17774..."
4,MA0 4,-2.477090,34.627656,MA,"POLYGON ((-3.33455 34.65145, -3.24272 34.81806..."
5,MA0 5,-7.387105,33.387598,MA,"POLYGON ((-8.05472 33.05307, -8.24447 33.21048..."
6,MA0 6,-5.613067,35.628486,MA,"POLYGON ((-6.15008 35.20700, -6.08153 35.39569..."
7,MA0 7,-10.715736,28.628010,MA,"POLYGON ((-12.91708 27.95319, -12.57097 27.994..."
8,MA0 8,-8.260102,32.234558,MA,"POLYGON ((-8.83512 31.08244, -9.02452 31.33669..."
9,MA0 9,-4.636334,34.173731,MA,"POLYGON ((-5.74076 33.44816, -5.49599 33.41281..."


In [45]:
gen_all_names=list(n.generators.index)
for i in range(len(gen_all_names)):
    gen_all_names[i]=gen_all_names[i].split(" ")[2]

gen_unique_names=list(set(gen_all_names))

gen_unique_names.remove("load")

In [46]:
selected_cols = ['p_nom_max', 'weight', 'p_nom', 'capital_cost', 'efficiency',
       'p_nom_min', 'marginal_cost', 'p_nom_extendable', 'carrier', 
       ]


In [47]:
def get_values_df(gen_unique_names,selected_cols,network):
    multi_index= pd.MultiIndex.from_product([gen_unique_names, selected_cols],
    names=['carrier','parameter'])
    param_bus_value_df = pd.DataFrame( index=multi_index,columns=gpd_bus_regions.name)

    for carrier_in_unique in gen_unique_names:
        generator_network=network.generators.copy()
        # carrier sorted value from generator_network
        carrier_df=generator_network[generator_network["carrier"] == carrier_in_unique]
        # arranging according to geopandas bus regions
        temp_name_series = pd.Series([0]*len(gpd_bus_regions), index=gpd_bus_regions.name, name="bus")
        # merging both to get all params df
        merged_gen_df=carrier_df.merge(temp_name_series, left_on="bus", right_on="name", how="right")
        # putting it in the param_bus_value_df 
        for param in selected_cols:
            param_bus_value_df.loc[(carrier_in_unique, param)]=list(merged_gen_df[param])

    param_bus_value_df.replace([np.inf, -np.inf,np.nan], 0, inplace=True)
    return param_bus_value_df
    

In [48]:
# # def get_p_nom(param_bus_value_df):
# #     p_nom_df=param_bus_value_df.loc[(slice(None),['p_nom']),:]
# #     p_nom_df=p_nom_df.reset_index()
# #     p_nom_df=p_nom_df.drop(columns=["parameter"])
# #     p_nom_df=p_nom_df.rename(columns={"level_0":"carrier"})
# #     p_nom_df=p_nom_df.set_index("carrier")
# #     return p_nom_df
# # def get_capacity(param_bus_value_df):
# #     pass

# def get_values_df_new(gen_unique_names,selected_cols,network):
#     multi_index= pd.MultiIndex.from_product([gen_unique_names, selected_cols],
#     names=['carrier','parameter'])
#     param_bus_value_df = pd.DataFrame( index=multi_index,columns=gpd_bus_regions.name)
#     fn_list = [get_p_nom, get_capacity]
#     for carrier_in_unique in gen_unique_names:
#         generator_network=network.generators.copy()
#         # carrier sorted value from generator_network
#         carrier_df=generator_network[generator_network["carrier"] == carrier_in_unique]
#         # arranging according to geopandas bus regions
#         temp_name_series = pd.Series([0]*len(gpd_bus_regions), index=gpd_bus_regions.name, name="bus")
#         # merging both to get all params df
#         merged_gen_df=carrier_df.merge(temp_name_series, left_on="bus", right_on="name", how="right")
#         # putting it in the param_bus_value_df 
#         for fn in fn_list:
#             param_bus_value_df.loc[(carrier_in_unique, param)]=fn(merged_gen_df)
#         print(merged_gen_df)

#         # param_bus_value_df[(carrier_in_unique), :] = (merged_gen_df        [selected_cols])


        

#     param_bus_value_df.replace([np.inf, -np.inf,np.nan], 0, inplace=True)
#     return param_bus_value_df

In [49]:
param_bus_value_df=get_values_df(gen_unique_names,selected_cols,n)

In [50]:
param_bus_value_df

name                     MA0 0 MA0 1         MA0 2     MA0 3 MA0 4   
carrier parameter                                                    
oil     p_nom_max            0     0             0         0     0  \
        weight               0     0           0.0         0     0   
        p_nom                0     0    277.900649         0     0   
        capital_cost         0     0  37870.807407         0     0   
        efficiency           0     0          0.35         0     0   
...                        ...   ...           ...       ...   ...   
ror     efficiency           0     0             0       0.9     0   
        p_nom_min            0     0             0       0.0     0   
        marginal_cost        0     0             0  0.010903     0   
        p_nom_extendable     0     0             0     False     0   
        carrier              0     0             0       ror     0   

name                             MA0 5         MA0 6         MA0 7 MA0 8 MA0 9  
carrier parameter                                                               
oil     p_nom_max                    0             0             0     0     0  
        weight                     0.0           0.0           0.0     0     0  
        p_nom              1114.381601    134.318647    211.386221     0     0  
        capital_cost      37870.807407  37870.807407  37870.807407     0     0  
        efficiency                0.35          0.35          0.35     0     0  
...                                ...           ...           ...   ...   ...  
ror     efficiency                 0.9             0             0     0     0  
        p_nom_min                  0.0             0             0     0     0  
        marginal_cost          0.00906             0             0     0     0  
        p_nom_extendable         False             0             0     0     0  
        carrier                    ror             0             0     0     0  

[63 rows x 10 columns]

In [51]:
opts = dict(
        xaxis=None,
        yaxis=None,
        active_tools=['pan', 'wheel_zoom']
    )

In [52]:
gpd_bus_regions["mdd"]=list(param_bus_value_df.loc[("oil", "p_nom")].values)

In [53]:
plot_area=gpd_bus_regions.hvplot(
    geo=True,
     tiles='OSM', alpha=1, 
                            hover_cols=['name'],
                              width=800, height=600,c="mdd").opts(**opts)


In [54]:
gpd_bus_regions

,name,x,y,country,geometry,mdd
0,MA0 0,-4.497494,32.115008,MA,"POLYGON ((-3.74821 31.36056, -3.76156 31.31018...",0.000000
1,MA0 1,-9.393582,30.564976,MA,"MULTIPOLYGON (((-10.57073 28.10503, -10.27730 ...",0.000000
2,MA0 2,-5.884074,34.243265,MA,"MULTIPOLYGON (((-6.42646 33.26058, -6.13605 33...",277.900649
3,MA0 3,-6.568445,32.124369,MA,"POLYGON ((-7.70016 30.91545, -7.63946 31.17774...",0.000000
4,MA0 4,-2.477090,34.627656,MA,"POLYGON ((-3.33455 34.65145, -3.24272 34.81806...",0.000000
5,MA0 5,-7.387105,33.387598,MA,"POLYGON ((-8.05472 33.05307, -8.24447 33.21048...",1114.381601
6,MA0 6,-5.613067,35.628486,MA,"POLYGON ((-6.15008 35.20700, -6.08153 35.39569...",134.318647
7,MA0 7,-10.715736,28.628010,MA,"POLYGON ((-12.91708 27.95319, -12.57097 27.994...",211.386221
8,MA0 8,-8.260102,32.234558,MA,"POLYGON ((-8.83512 31.08244, -9.02452 31.33669...",0.000000
9,MA0 9,-4.636334,34.173731,MA,"POLYGON ((-5.74076 33.44816, -5.49599 33.41281...",0.000000


In [55]:
x=list(gpd_bus_regions["x"].values)
y=list(gpd_bus_regions["y"].values)
points = gpd_bus_regions.copy()
points.geometry = gpd.points_from_xy(x, y, crs=4326)
points["mdd"]=list(param_bus_value_df.loc[("oil", "p_nom")].values)

marker_size = points["mdd"] / points["mdd"].max() * 300
node_plot = points.hvplot(
            geo=True,
            hovercols=["name"],
            size=300,
            s = marker_size,
            c="mdd",
            alpha=0.7
        ).opts(**opts)



In [56]:
points

,name,x,y,country,geometry,mdd
0,MA0 0,-4.497494,32.115008,MA,POINT (-4.49749 32.11501),0.000000
1,MA0 1,-9.393582,30.564976,MA,POINT (-9.39358 30.56498),0.000000
2,MA0 2,-5.884074,34.243265,MA,POINT (-5.88407 34.24327),277.900649
3,MA0 3,-6.568445,32.124369,MA,POINT (-6.56845 32.12437),0.000000
4,MA0 4,-2.477090,34.627656,MA,POINT (-2.47709 34.62766),0.000000
5,MA0 5,-7.387105,33.387598,MA,POINT (-7.38711 33.38760),1114.381601
6,MA0 6,-5.613067,35.628486,MA,POINT (-5.61307 35.62849),134.318647
7,MA0 7,-10.715736,28.628010,MA,POINT (-10.71574 28.62801),211.386221
8,MA0 8,-8.260102,32.234558,MA,POINT (-8.26010 32.23456),0.000000
9,MA0 9,-4.636334,34.173731,MA,POINT (-4.63633 34.17373),0.000000


In [57]:
# s=hv.render(plot_area,backend='bokeh')
# show(s)

In [58]:
nLines=n.lines.copy()



In [59]:
n.lines

,bus0,bus1,num_parallel,length,type,s_max_pu,s_nom,capital_cost,s_nom_extendable,s_nom_min,...,build_year,lifetime,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu
Line,,,,,,,,,,,,,,,,,,,,,
1,MA0 0,MA0 3,1.157895,217.164436,Al/St 240/40 4-bundle 380.0,0.7,1966.224077,8955.874833,True,1138.340255,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0
2,MA0 1,MA0 7,0.578947,315.480486,Al/St 240/40 4-bundle 380.0,0.7,983.112038,13007.253003,True,983.112038,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0
3,MA0 1,MA0 8,4.026316,261.174202,Al/St 240/40 4-bundle 380.0,0.7,6837.097358,10770.218843,True,6837.097358,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0
4,MA0 2,MA0 3,0.289474,302.721936,Al/St 240/40 4-bundle 380.0,0.7,491.556019,12481.960423,True,491.556019,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0
5,MA0 2,MA0 4,0.289474,395.496104,Al/St 240/40 4-bundle 380.0,0.7,491.556019,16309.170689,True,491.556019,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0
6,MA0 2,MA0 5,4.315789,210.478096,Al/St 240/40 4-bundle 380.0,0.7,7328.653377,8679.509637,True,7328.653377,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0
7,MA0 2,MA0 6,4.315789,194.714948,Al/St 240/40 4-bundle 380.0,0.7,7328.653377,8030.834595,True,7328.653377,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0
8,MA0 2,MA0 9,4.605263,144.723286,Al/St 240/40 4-bundle 380.0,0.7,7820.209396,5967.744495,True,7820.209396,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0
9,MA0 3,MA0 5,2.026316,196.988781,Al/St 240/40 4-bundle 380.0,0.7,3440.892134,8122.720974,True,1992.095446,...,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0


In [60]:

nLines["Total Capacity (GW)"] = nLines.s_nom_opt.clip(lower=1e-3)
nLines["Reinforcement (GW)"] =  nLines.s_nom_opt.clip(lower=1e-3) - nLines.s_nom.clip(lower=1e-3)
nLines["Original Capacity (GW)"] = nLines.s_nom.clip(lower=1e-3)
nLines["Maximum Capacity (GW)"] = nLines.s_nom_min.clip(lower=1e-3)



In [61]:
nLines["Total Capacity (GW)"]

Line
1     1138.340255
2     2685.791713
3     6837.097358
4      491.556019
5      491.556019
6     7328.653377
7     7328.653377
8     7820.209396
9     1992.095446
10    1138.340255
11     569.170127
12    4155.882708
13    8803.321435
Name: Total Capacity (GW), dtype: float64

In [62]:
G = nx.from_pandas_edgelist(nLines, 'bus0', 'bus1',edge_attr= ["Total Capacity (GW)", "Reinforcement (GW)", "Original Capacity (GW)", "Maximum Capacity (GW)"])


In [63]:
G.edges

EdgeView([('MA0 0', 'MA0 3'), ('MA0 3', 'MA0 2'), ('MA0 3', 'MA0 5'), ('MA0 3', 'MA0 8'), ('MA0 3', 'MA0 9'), ('MA0 1', 'MA0 7'), ('MA0 1', 'MA0 8'), ('MA0 8', 'MA0 5'), ('MA0 2', 'MA0 4'), ('MA0 2', 'MA0 5'), ('MA0 2', 'MA0 6'), ('MA0 2', 'MA0 9'), ('MA0 4', 'MA0 9')])

In [64]:
nodes_pos_dict={}

for point in gpd_bus_regions.itertuples():
    nodes_pos_dict[point.name]=(point.x,point.y)

In [65]:
nodes_pos_dict

{'MA0 0': (-4.4974944444444445, 32.115008333333336),
 'MA0 1': (-9.393582142857142, 30.56497619047619),
 'MA0 2': (-5.884074428571429, 34.24326528571429),
 'MA0 3': (-6.568445185185185, 32.12436851851852),
 'MA0 4': (-2.4770897435897434, 34.627656410256414),
 'MA0 5': (-7.387105238095238, 33.38759761904762),
 'MA0 6': (-5.613066568627452, 35.628486274509804),
 'MA0 7': (-10.715735833333333, 28.62801),
 'MA0 8': (-8.260101984126985, 32.23455761904762),
 'MA0 9': (-4.636334408602151, 34.173730645161285)}

In [66]:
scale = pd.Series(nx.get_edge_attributes(G,"Reinforcement (GW)" )).max() / 10

In [67]:
network_lines_plot=hvnx.draw_networkx_edges(G, pos=nodes_pos_dict,edge_color="Reinforcement (GW)",responsive=True,
            inspection_policy="edges",node_color='#A0C0E2',geo=True,edge_width=hv.dim("Reinforcement (GW)")/scale ).opts(**opts)



In [68]:
s=hv.render(plot_area*node_plot*network_lines_plot,backend='bokeh')
show(s)

In [69]:
time_s_data=n.generators_t["p"].copy()


In [70]:
def get_unique_carriers(df):
    all_cols=df.columns
    split_cols= []
    for k in all_cols:
        split_cols.append(k.split(" ")[-1])
    return list(set(split_cols))

unique_carriers=get_unique_carriers(time_s_data)
print(unique_carriers)

['oil', 'load', 'CCGT', 'coal', 'onwind', 'OCGT', 'solar', 'ror']


In [71]:
resultant_df=pd.DataFrame(0,columns=unique_carriers,index=time_s_data.index,)


In [72]:

for carrier in unique_carriers:
    # time_s_data[carrier]=0
    for bus_carrier in time_s_data.columns:
        if carrier in bus_carrier.split(" ") :
            resultant_df[carrier]+=time_s_data[bus_carrier]

In [73]:
kwargs = dict(
        stacked=True,
        line_width=0,
        xlabel='',
        width=800,
        height=350,
        hover=False,
        legend='top'
    )

area_plot=resultant_df.hvplot.area(**kwargs, ylabel="Supply [GW]")


In [74]:
s=hv.render(area_plot,backend='bokeh')
show(s)

In [75]:
lines_plot=n.lines_t["p0"].hvplot.area(**kwargs, ylabel="Flow [GW]")

In [76]:
s=hv.render(lines_plot,backend='bokeh')
show(s)

In [77]:
storage_df=n.storage_units_t["spill"].copy()

storage_plot=storage_df.hvplot.area(**kwargs, ylabel="Storage [GW]")

In [78]:
s=hv.render(storage_plot,backend='bokeh')
show(s)

In [90]:
n.links_t["p0"].columns

Index(['MA0 0 H2 Electrolysis', 'MA0 1 H2 Electrolysis',
       'MA0 2 H2 Electrolysis', 'MA0 3 H2 Electrolysis',
       'MA0 4 H2 Electrolysis', 'MA0 5 H2 Electrolysis',
       'MA0 6 H2 Electrolysis', 'MA0 7 H2 Electrolysis',
       'MA0 8 H2 Electrolysis', 'MA0 9 H2 Electrolysis', 'MA0 0 H2 Fuel Cell',
       'MA0 1 H2 Fuel Cell', 'MA0 2 H2 Fuel Cell', 'MA0 3 H2 Fuel Cell',
       'MA0 4 H2 Fuel Cell', 'MA0 5 H2 Fuel Cell', 'MA0 6 H2 Fuel Cell',
       'MA0 7 H2 Fuel Cell', 'MA0 8 H2 Fuel Cell', 'MA0 9 H2 Fuel Cell',
       'MA0 0 battery charger', 'MA0 1 battery charger',
       'MA0 2 battery charger', 'MA0 3 battery charger',
       'MA0 4 battery charger', 'MA0 5 battery charger',
       'MA0 6 battery charger', 'MA0 7 battery charger',
       'MA0 8 battery charger', 'MA0 9 battery charger',
       'MA0 0 battery discharger', 'MA0 1 battery discharger',
       'MA0 2 battery discharger', 'MA0 3 battery discharger',
       'MA0 4 battery discharger', 'MA0 5 battery discharger',


In [ ]:
def get_links_unique_cols(pypsa_network):
    all_cols=pypsa_network.links_t["p0"].columns
    split_cols= []
    for k in all_cols:
        split_cols.append(k.split(" ")[-1])
    return list(set(split_cols))

In [ ]:
def get_links_df(pypsa_network,links_t_key):
    links_t_df=pypsa_network.links_t[links_t_key]
    unique_cols=get_links_unique_cols(pypsa_network)
    resultant_df=pd.DataFrame(0,columns=unique_cols,index=links_t_df.index)

    for carrier in unique_cols:
        for links_carrier in links_t_df.columns:
            if carrier in links_carrier.split(" ") :
                resultant_df[carrier]+=links_t_df[bus_carrier]
    return resultant_df
